In [1]:
import time
start_time = time.time()

import cv2
import sqlite3
import numpy as np
from imgbeddings import imgbeddings
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

# Database and file paths (adjust as needed)
db_name = 'C:\\Users\\asus\\Desktop\\test\\faces2.db'
table_name = 'images'
column_name = 'image_data'
casPath = "C:\\Users\\asus\\Desktop\\test\\haarcascade_frontalface_default.xml"

# Connect to database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
faceCascade = cv2.CascadeClassifier(casPath)

def retrieve_face_data(image_id, cursor, conn):
    query = f"SELECT {column_name} FROM {table_name} WHERE image_id = ?"
    cursor.execute(query, (image_id,))

    image_data = cursor.fetchone()
    image_data = np.frombuffer(image_data[0], dtype=np.uint8)  # Access first element of row
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    return image


def compare_faces(image1, image2):
    image1_array = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2_array = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    image1_pil = Image.fromarray(image1_array)
    image2_pil = Image.fromarray(image2_array)

    embedder = imgbeddings()

    # Generate embeddings for the face images
    image1_embedding = embedder.to_embeddings([image1_pil])
    image2_embedding = embedder.to_embeddings([image2_pil])

    # Calculate the cosine similarity between the embeddings
    similarity = cosine_similarity(image1_embedding, image2_embedding)

    return similarity[0][0]


# Example usage (assuming you have a cursor and connection object)
cap = cv2.VideoCapture(0)

while True:
            
        ret, frame = cap.read()

        # Check if the frame is received correctly?
        if not ret:
            print("Error: Cannot receive frame from webcam")
            break

        # Detect faces using the cascade classifier
        faces = faceCascade.detectMultiScale(
            frame,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(150, 150),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # For each face in the frame:
        for (x, y, w, h) in faces:
            cv2.putText(frame, 'please click c', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
            roi = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow('Webcam', frame)
        key = cv2.waitKey(1)
        if key == ord('c'):       
            cv2.imwrite(filename='saved_img.jpg',img = roi )
            cap.release()
            img_new = cv2.imread('saved_img.jpg')
            image2 = cv2.fastNlMeansDenoising(img_new)
            cv2.waitKey(1)
            cv2.destroyAllWindows()
            break

# Retrieve face data from the database
cursor.execute(f"SELECT image_id FROM {table_name}")
image_ids = cursor.fetchall()

for image_id in image_ids:
    image1 = retrieve_face_data(image_id[0], cursor, conn)
    similarity = compare_faces(image1, image2)

    if similarity > 0.9:
        cursor.execute(f"SELECT image_name FROM {table_name} WHERE image_id = ?", (image_id[0],))
        result = cursor.fetchone()
        if result:
            image_name = result[0]
            print(f"Similar image found in the database! Name: {image_name}")
        break

# Close the database connection
conn.close()

end_time = time.time()  # Record the end time after the code block

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

E:\ana\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


Similar image found in the database! Name: ali
Execution time: 190.37 seconds
